## Assignment 3: $k$ Nearest Neighbor and $k$ Means Clustering

## **Do three questions.**

`! git clone https://www.github.com/DS3001/assignment3`

**Q1.** This question is a case study for $k$ nearest neighbor. The target variable `y` is `Purchase` and the features of interest are `Age` and `AnnualSalary`.

1. Load the `./data/car_data.csv` data. Look at the head and dimensions of the data.
2. Summarize the variables (`User.ID`, `Gender`, `Age`, `AnnualSalary`, `Purchase`). Are there any missings to clean? Convert the `Gender` categorical variable into a dummy variable that takes the value 0 for male and 1 for female. Create a matrix $X$ of predictors including `Age` and `AnnualSalary`, and an outcome $y$ equaling `Purchase`.
3. MaxMin-normalize `Age` and `AnnualSalary` in `X`.
4. Split the sample into a ~80% training dataset and a ~20% testing dataset.
5. Treat this as a classification problem: The model is supposed to predict 0 or 1 for each customer, classifying them as a purchaser or non-purchaser. Use sklearn to determine the optimal number of neighbors $k$ to use.
6. Run the model for the optimal number of neighbors on the testing data. Cross tabulate the predicted outcomes against the actual outcomes; this is called a **confusion matrix**. How often does the model predict a sale when one fails to occur? How often does the model predict no sale when one does occur? Overall, does it provide accurate predictions?
7. Now, compute confusion matrices separately for men and women, as in part 6. Does the model make more accurate predictions for one sex or the other? Explain. (Performance of algorithms on population subgroups is a growing topic in data science.)

In [3]:
# Question 1 Code

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor as KNR

url1 = "https://raw.githubusercontent.com/3agleman/assignment3/main/data/car_data.csv"
carData = pd.read_csv(url1)

# 1)
#print("PART 1")
#print(carData.size)
#print(carData.head)

# 2)
#print("PART 2")
#print(carData.describe())

# No missing values here, from what I can tell
#print(carData['User ID'].unique())
#print(carData['Gender'].unique())
#print(carData['Age'].unique())
#print(carData['AnnualSalary'].unique())
#print(carData['Purchased'].unique())

carData['Gender'] = carData['Gender'].replace("Male", 0)
carData['Gender'] = carData['Gender'].replace("Female", 1)

y = carData['Purchased']
X = carData.loc[:,['Age','AnnualSalary']]


# 3)
def maxMinNormalize(num):
  num = (num-min(num))/((max(num)-min(num)))
  return(num)

X = X.apply(maxMinNormalize)

# 4)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 100)

# 5)
k_max = 100
k_array = np.arange(1,k_max)
SSE_train = np.zeros(k_max)
SSE_test = np.zeros(k_max)

for k in range(k_max):
  model = KNR(n_neighbors = k+1)
  fitted_model = model.fit(X_train, y_train)
  y_hat = fitted_model.predict(X_test)
  SSE_test[k] = np.sum((y_test - y_hat)**2)
  y_hat = fitted_model.predict(X_train)
  SSE_train[k] = np.sum((y_train - y_hat)**2)

SSE_min = np.min(SSE_train)
min_index = np.where(SSE_train == SSE_min)
k_star = k_array[min_index]
print("Optimal k: ", k_star)
# Apparently optimal k is 1?

# 6)
model = KNR(n_neighbors = k_star[0])
classifier_model = model.fit(X_train, y_train)

N_x = 100
N_y = 100
total = N_x*N_y

grid_x = np.linspace(0,1,N_x)
grid_y = np.linspace(0,1,N_y)
xs, ys = np.meshgrid(grid_x, grid_y)

X = xs.reshape(total)
Y = ys.reshape(total)

x_hat = pd.DataFrame({"Age":X,"AnnualSalary":Y})
y_hat = classifier_model.predict(x_hat)
x_hat['Predicted purchase'] = y_hat


confusionMatrix = pd.DataFrame(data = y).rename(columns = {"Purchased":"y"})
confusionMatrix['y_hat'] = x_hat['Predicted purchase']

falsePositive = (confusionMatrix['y'] == 0) & (confusionMatrix['y_hat'] == 1)
numFalsePositive = falsePositive.sum()
falseNegative = (confusionMatrix['y'] == 1) & (confusionMatrix['y_hat'] == 0)
numFalseNegative = falseNegative.sum()
total_all = confusionMatrix['y'].count()

falsePositiveRate = numFalsePositive/total_all
falseNegativeRate = numFalsePositive/total_all
totalErrorRate = (numFalsePositive + numFalseNegative)/total_all
# print(totalErrorRate)

# It seems that the total error rate for this model is 46.6%. So, overall, not very accurate, almost 50/50.

# 7)
onlyMen = (carData['Gender'] == 0)
onlyWomen = (carData['Gender'] == 1)

menData = carData.loc[onlyMen]
womenData = carData.loc[onlyWomen]

yMen = menData['Purchased']
XMen = menData.loc[:,['Age','AnnualSalary']]
XMen = XMen.apply(maxMinNormalize)
XMen_train, XMen_test, yMen_train, yMen_test = train_test_split(XMen, yMen, test_size = .2, random_state = 100)
modelMen = KNR(n_neighbors = 1)
classifier_model = modelMen.fit(XMen_train, yMen_train)

N_xMen = 100
N_yMen = 100
total = N_xMen*N_yMen

grid_xMen = np.linspace(0,1,N_xMen)
grid_yMen = np.linspace(0,1,N_yMen)
xsMen, ysMen = np.meshgrid(grid_xMen, grid_yMen)

XMen = xsMen.reshape(total)
YMen = ysMen.reshape(total)

xMen_hat = pd.DataFrame({"Age":XMen,"AnnualSalary":YMen})
yMen_hat = classifier_model.predict(xMen_hat)
xMen_hat['Predicted purchase'] = yMen_hat

confusionMatrix = pd.DataFrame(data = yMen).rename(columns = {"Purchased":"y"})
confusionMatrix['y_hat'] = xMen_hat['Predicted purchase']

falsePositive = (confusionMatrix['y'] == 0) & (confusionMatrix['y_hat'] == 1)
numFalsePositive = falsePositive.sum()
falseNegative = (confusionMatrix['y'] == 1) & (confusionMatrix['y_hat'] == 0)
numFalseNegative = falseNegative.sum()
total_all = confusionMatrix['y'].count()

falsePositiveRate = numFalsePositive/total_all
falseNegativeRate = numFalsePositive/total_all
totalErrorRate = (numFalsePositive + numFalseNegative)/total_all
print("Men: ", totalErrorRate)

#############################################
yWomen = womenData['Purchased']
XWomen = womenData.loc[:,['Age','AnnualSalary']]
XWomen = XWomen.apply(maxMinNormalize)
XWomen_train, XWomen_test, yWomen_train, yWomen_test = train_test_split(XWomen, yWomen, test_size = .2, random_state = 100)
modelWomen = KNR(n_neighbors = 1)
classifier_model = modelWomen.fit(XWomen_train, yWomen_train)

N_xWomen = 100
N_yWomen = 100
total = N_xWomen*N_yWomen

grid_xWomen = np.linspace(0,1,N_xWomen)
grid_yWomen = np.linspace(0,1,N_yWomen)
xsWomen, ysWomen = np.meshgrid(grid_xWomen, grid_yWomen)

XWomen = xsWomen.reshape(total)
YWomen = ysWomen.reshape(total)

xWomen_hat = pd.DataFrame({"Age":XWomen,"AnnualSalary":YWomen})
yWomen_hat = classifier_model.predict(xWomen_hat)
xWomen_hat['Predicted purchase'] = yWomen_hat

confusionMatrix = pd.DataFrame(data = yWomen).rename(columns = {"Purchased":"y"})
confusionMatrix['y_hat'] = xWomen_hat['Predicted purchase']

falsePositive = (confusionMatrix['y'] == 0) & (confusionMatrix['y_hat'] == 1)
numFalsePositive = falsePositive.sum()
falseNegative = (confusionMatrix['y'] == 1) & (confusionMatrix['y_hat'] == 0)
numFalseNegative = falseNegative.sum()
total_all = confusionMatrix['y'].count()

falsePositiveRate = numFalsePositive/total_all
falseNegativeRate = numFalsePositive/total_all
totalErrorRate = (numFalsePositive + numFalseNegative)/total_all
print("Women: ", totalErrorRate)

# There doesn't appear to be a substantial difference in error rates here,
# although the model with only men is ~1.4% more accurate than the model with only women.

Optimal k:  [1]
Men:  0.4524793388429752
Women:  0.4689922480620155


**Q2.** This question is a case study for $k$ nearest neighbor The target variable `y` is `price` and the features are `year` and `mileage`.

1. Load the `./data/USA_cars_datasets.csv`. Keep the following variables and drop the rest: `price`, `year`, `mileage`. Are there any `NA`'s to handle? Look at the head and dimensions of the data.
2. Maxmin normalize `year` and `mileage`.
3. Split the sample into ~80% for training and ~20% for evaluation.
4. Use the $k$NN algorithm for regression and the training data to predict `price` using `year` and `mileage` for the test set for $k=3,10,25,50,100,300$. For each value of $k$, compute the Sum of Squared Error and make a scatterplot showing the test value plotted against the predicted value. What patterns do you notice as you increase $k$?
5. Determine the optimal $k$ for these data.
6. Describe what happened in the plots of predicted versus actual prices as $k$ varied, taking your answer into part 6 into account. (Hint: Use the words "underfitting" and "overfitting".)

In [22]:
# Question 2 Code
import seaborn as sns

# 1)
url2 = 'https://raw.githubusercontent.com/3agleman/assignment3/main/data/USA_cars_datasets.csv'
USAcarsAll = pd.read_csv(url2)

USAcars = USAcarsAll.loc[:,['price','year','mileage']]

#print(pd.isna(USAcars['price']).sum())
#print(pd.isna(USAcars['year']).sum())
#print(pd.isna(USAcars['mileage']).sum())
# No NAs here.

#print(USAcars.head)
#print(USAcars.size)

# 2)
y = USAcars['price']
X = USAcars.loc[:,['year','mileage']]
X = X.apply(maxMinNormalize)

# 3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 100)

# 4)

def SSE_check(k, data = USAcars):

  model = KNR(n_neighbors = k)
  fitted_model = model.fit(X_train, y_train)
  y_hat = fitted_model.predict(X_test)
  SSE_test = np.sum((y_test - y_hat)**2)
  print("SSE_test for k ", k, ": ", SSE_test)
  y_hat = fitted_model.predict(X_train)
  SSE_train = np.sum((y_train - y_hat)**2)
  print("SSE_train for k ", k, ": ", SSE_train)

def model_check(k, data = USAcars):
  y = USAcars['price']
  X = USAcars.loc[:,['year','mileage']]
  X = X.apply(maxMinNormalize)
  model = KNR(n_neighbors = k)
  fitted_model = model.fit(X, y)
  N_x = 100
  N_y = 100
  total = N_x*N_y
  grid_x = np.linspace(0,1,N_x)
  grid_y = np.linspace(0,1,N_y)
  xs, ys = np.meshgrid(grid_x, grid_y)
  X = xs.reshape(total)
  Y = ys.reshape(total)
  x_hat = pd.DataFrame({"year":X,"mileage":Y})
  y_hat = fitted_model.predict(x_hat)
  x_hat['Predicted price'] = y_hat
  x_hat['Actual price'] = y
  k_scatter = sns.scatterplot(data = x_hat, x = 'Actual price', y = 'Predicted price')


#model_check(3)
SSE_check(3)
#model_check(10)

#model_check(25)

#model_check(50)

#model_check(100)


#It seems that the models are uniformly inaccurate regardless of K.

# 5)
k_max = 100
k_array = np.arange(1,k_max)
SSE_train = np.zeros(k_max)
SSE_test = np.zeros(k_max)

for k in range(k_max):
  model = KNR(n_neighbors = k+1)
  fitted_model = model.fit(X_train, y_train)
  y_hat = fitted_model.predict(X_test)
  SSE_test[k] = np.sum((y_test - y_hat)**2)
  y_hat = fitted_model.predict(X_train)
  SSE_train[k] = np.sum((y_train - y_hat)**2)

SSE_min = np.min(SSE_test)
min_index = np.where(SSE_test == SSE_min)
k_star = k_array[min_index]
print("Optimal k: ", k_star)
# It looks like the optimal K is 77, according to the test data.
# I also tried looking at the minimum SSE from the training data, but it said it
# was 1 too, so I'm unsure if that's very accurate.

# 6)
# It's hard to tell since all the scatterplots from question 5 essentially looked the same,
# but I'd say that most of the results I had, regardless of k, were underfitted. The predicted y-values
# were inconsistent with the actual values, so I'd need to figure out what exactly went wrong before proceeding.
# The fact that the optimal k is 77 based on the test data could be a clue.

SSE_test for k  3 :  63223286609.11111
SSE_train for k  3 :  144929703627.88885
Optimal k:  [77]


**Q3.** This question is a case study for $k$ nearest neighbor, particularly variable selection. The data for the question include (summaries reproduced here directly from the codebook):

- age: age of the patient (years)
- anaemia: decrease of red blood cells or hemoglobin (boolean)
- high blood pressure: if the patient has hypertension (boolean)
- creatinine phosphokinase (CPK): level of the CPK enzyme in the blood (mcg/L)
- diabetes: if the patient has diabetes (boolean)
- ejection fraction: percentage of blood leaving the heart at each contraction (percentage)
- platelets: platelets in the blood (kiloplatelets/mL)
- sex: woman or man (binary)
- serum creatinine: level of serum creatinine in the blood (mg/dL)
- serum sodium: level of serum sodium in the blood (mEq/L)
- smoking: if the patient smokes or not (boolean)
- time: follow-up period (days)
- death event: if the patient deceased during the follow-up period (boolean)

1. Load the `./data/heart_failure_clinical_records_dataset.csv`. Are there any `NA`'s to handle? use `.drop()` to remove `time` from the dataframe.
2. Make a correlation matrix. What variables are strongly associated with a death event?
3. For the dummy variables `anaemia`, `diabetes`, `high_blood_pressure`, `sex`, and `smoking`, compute a summary table of `DEATH_EVENT` grouped by the variable. For which variables does a higher proportion of the population die when the variable takes the value 1 rather than 0?
4. On the basis of your answers from 2 and 3, build a matrix $X$ of the variables you think are most predictive of a death, and a variable $y$ equal to `DEATH_EVENT`.
5. Maxmin normalize all of the variables in `X`.
6. Split the sample into ~80% for training and ~20% for evaluation. (Try to use the same train/test split for the whole question, so that you're comparing apples to apples in the questions below.).
7. Determine the optimal number of neighbors for a $k$NN regression for the variables you selected.
8. OK, do steps 5 through 7 again, but use all of the variables (except `time`). Which model has a lower Sum of Squared Error? Which would you prefer to use in practice, if you had to predict `DEATH_EVENT`s? If you play with the selection of variables, how much does the SSE change for your fitted model on the test data? Are more variables better, in this case? Explain your findings.

**Q4.** Let's do some very basic computer vision. We're going to import the MNIST handwritten digits data and $k$NN to predict values (i.e. "see/read").

1. To load the data, run the following code in a chunk:
```
from keras.datasets import mnist
df = mnist.load_data('minst.db')
train,test = df
X_train, y_train = train
X_test, y_test = test
```
The `y_test` and `y_train` vectors, for each index `i`, tell you want number is written in the corresponding index in `X_train[i]` and `X_test[i]`. The value of `X_train[i]` and `X_test[i]`, however, is a 28$\times$28 array whose entries contain values between 0 and 256. Each element of the matrix is essentially a "pixel" and the matrix encodes a representation of a number. To visualize this, run the following code to see the first ten numbers:
```
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000)
for i in range(5):
    print(y_test[i],'\n') # Print the label
    print(X_test[i],'\n') # Print the matrix of values
    plt.contourf(np.rot90(X_test[i].transpose())) # Make a contour plot of the matrix values
    plt.show()
```
OK, those are the data: Labels attached to handwritten digits encoded as a matrix.

2. What is the shape of `X_train` and `X_test`? What is the shape of `X_train[i]` and `X_test[i]` for each index `i`? What is the shape of `y_train` and `y_test`?
3. Use Numpy's `.reshape()` method to covert the training and testing data from a matrix into an vector of features. So, `X_test[index].reshape((1,784))` will convert the $index$-th element of `X_test` into a $28\times 28=784$-length row vector of values, rather than a matrix. Turn `X_train` into an $N \times 784$ matrix $X$ that is suitable for scikit-learn's kNN classifier where $N$ is the number of observations and $784=28*28$ (you could use, for example, a `for` loop).
4. Use the reshaped `X_test` and `y_test` data to create a $k$-nearest neighbor classifier of digit. What is the optimal number of neighbors $k$? If you can't determine this, play around with different values of $k$ for your classifier.
5. For the optimal number of neighbors, how well does your predictor perform on the test set?
6. So, this is how computers "see." They convert an image into a matrix of values, that matrix becomes a vector in a dataset, and then we deploy ML tools on it as if it was any other kind of tabular data. To make sure you follow this, invent a way to represent a color photo in matrix form, and then describe how you could convert it into tabular data. (Hint: RGB color codes provide a method of encoding a numeric value that represents a color.)

**Q5.** This question is a case study for $k$ means clustering.

1. Load the `airbnb_hw.csv` data. Clean `Price` along with `Beds`, `Number of Reviews`, and `Review Scores Rating`.
2. Maxmin normalize the data and remove any `nan`'s (`KMeans` from `sklearn` doesn't accept `nan` input).
3. Use `sklearn`'s `KMeans` module to cluster the data by `Beds`, `Number of Reviews`, and `Review Scores Rating` for `k=6`.
4. Use `seaborn`'s `.pairplot()` to make a grid of scatterplots that show how the clustering is carried out in multiple dimensions.
5. Use `.groupby` and `.describe` to compute the average price for each cluster. Which clusters have the highest rental prices?
6. Use a scree plot to pick the number of clusters and repeat steps 4 and 5.

**Q6.** This is a question about $k$ means clustering. We want to investigate how adjusting the "noisiness" of the data impacts the quality of the algorithm and the difficulty of picking $k$.

1. Run the code below, which creates four datasets: `df0_125`, `df0_25`, `df0_5`, `df1_0`, and `df2_0`. Each data set is created by increasing the amount of `noise` (standard deviation) around the cluster centers, from `0.125` to `0.25` to `0.5` to `1.0` to `2.0`.

```
import numpy as np
import pandas as pd

def createData(noise,N=50):
    np.random.seed(100) # Set the seed for replicability
    # Generate (x1,x2,g) triples:
    X1 = np.array([np.random.normal(1,noise,N),np.random.normal(1,noise,N)])
    X2 = np.array([np.random.normal(3,noise,N),np.random.normal(2,noise,N)])
    X3 = np.array([np.random.normal(5,noise,N),np.random.normal(3,noise,N)])
    # Concatenate into one data frame
    gdf1 = pd.DataFrame({'x1':X1[0,:],'x2':X1[1,:],'group':'a'})
    gdf2 = pd.DataFrame({'x1':X2[0,:],'x2':X2[1,:],'group':'b'})
    gdf3 = pd.DataFrame({'x1':X3[0,:],'x2':X3[1,:],'group':'c'})
    df = pd.concat([gdf1,gdf2,gdf3],axis=0)
    return df

df0_125 = createData(0.125)
df0_25 = createData(0.25)
df0_5 = createData(0.5)
df1_0 = createData(1.0)
df2_0 = createData(2.0)
```

2. Make scatterplots of the $(X1,X2)$ points by group for each of the datasets. As the `noise` goes up from 0.125 to 2.0, what happens to the visual distinctness of the clusters?
3. Create a scree plot for each of the datasets. Describe how the level of `noise` affects the scree plot (particularly the presence of a clear "elbow") and your ability to definitively select a $k$.
4. Explain the intuition of the elbow, using this numerical simulation as an example.

**Q7.** We looked at computer vision with $k$NN in a previous question. Can $k$ means clustering correctly group digits, even if we don't know which symbols are which?

1. To load the data, run the following code in a chunk:
```
from keras.datasets import mnist
df = mnist.load_data('minst.db')
train,test = df
X_train, y_train = train
X_test, y_test = test
```
The `y_test` and `y_train` vectors, for each index `i`, tell you want number is written in the corresponding index in `X_train[i]` and `X_test[i]`. The value of `X_train[i]` and `X_test[i]`, however, is a 28$\times$28 array whose entries contain values between 0 and 256. Each element of the matrix is essentially a "pixel" and the matrix encodes a representation of a number. To visualize this, run the following code to see the first ten numbers:
```
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000)
for i in range(5):
    print(y_test[i],'\n') # Print the label
    print(X_test[i],'\n') # Print the matrix of values
    plt.contourf(np.rot90(X_test[i].transpose())) # Make a contour plot of the matrix values
    plt.show()
```
OK, those are the data: Labels attached to handwritten digits encoded as a matrix.

2. What is the shape of `X_train` and `X_test`? What is the shape of `X_train[i]` and `X_test[i]` for each index `i`? What is the shape of `y_train` and `y_test`?
3. Use Numpy's `.reshape()` method to covert the training and testing data from a matrix into an vector of features. So, `X_test[index].reshape((1,784))` will convert the $index$-th element of `X_test` into a $28\times 28=784$-length row vector of values, rather than a matrix. Turn `X_train` into an $N \times 784$ matrix $X$ that is suitable for scikit-learn's kNN classifier where $N$ is the number of observations and $784=28*28$ (you could use, for example, a `for` loop).
4. Use $k$ means clustering on the reshaped `X_test` data with `k=10`.  
5. Cross tabulate the cluster assignments with the true labels for the test set values. How good is the correspondence? What proportion of digits are clustered correctly? Which digits are the hardest to distinguish from one another? Can $k$MC recover the latent digits 0 to 9, without even knowing what those digits were?
6. If you use a scree plot to determine the number of clusters $k$, does it pick 10 (the true number of digits), or not? If it fails to pick $k=10$, which digits does it tend to combine into the same classification?

SOURCES AND REFERENCES USED:

Class Notes kNN

https://pandas.pydata.org/pandas-docs/stable/reference/frame.html

https://numpy.org/doc/stable/reference/routines.html

https://365datascience.com/tutorials/machine-learning-tutorials/overfitting-underfitting/